In [1]:
import pandas as pd
import itertools
import os

import similarityregression.PairwiseAlignment as pwsaln
import similarityregression.AlignmentTools as alntools
import similarityregression.PredictSimilarity as srpred

# Introduction

This notebook describes an example of how to parse protein sequences from CisBP into a form that can be used for Similarity Regression (training and scoring). Source data: 5 Homeodomain PBM experiments from CisBP (http://cisbp.ccbr.utoronto.ca/). Motif Similarity was calculated with MoSBAT (http://mosbat.ccbr.utoronto.ca/) as an example (E-score overlaps are used in the paper).

# Parse CisBP Data
## Read E-scores and calculate motif similarity (E-Score Overlaps)


In [2]:
escores = pd.read_csv('HomeodomainData/EScore.txt', index_col=0, delimiter='\t')
escores.columns = [x.split(':')[0] for x in escores.columns] #Relabel by moits
escores = escores.groupby(by=escores.columns, axis=1).mean() #Average E-scores for replicates

#Calculate E-score Overlaps
scorethresh = 0.45
EScoreOverlaps = {}

MIDs = list(escores.columns)
MIDs.sort()
for x,y in itertools.combinations(MIDs, 2):
    #print x, y
    maxN = max(len(set(escores[escores[x] >= scorethresh][x].index)), len(set(escores[escores[y] >= scorethresh][y].index)))
    x_maxN = set(escores[x].sort_values(ascending = False).iloc[:maxN].index)
    y_maxN = set(escores[y].sort_values(ascending = False).iloc[:maxN].index)
    escoreoverlap = 1.0*len(x_maxN.intersection(y_maxN))/maxN
    #print escoreoverlap, len(x_maxN.intersection(y_maxN)), maxN
    EScoreOverlaps[(x,y)] = escoreoverlap

# TF / Protein Information 
## Parse Protein Information to reference DBD alignment

In [3]:
proteins = pd.read_csv('HomeodomainData/prot_seq.txt', index_col=0, delimiter='\t')

motifs = []
with open('HomeodomainData/TF_Information.txt', 'r') as infile:
    h = infile.readline().strip().split('\t')
    for line in infile:
        line = line.strip().split('\t')
        data = line[:6]
        mids = line[6:]
        mids = [x[:-1] for x in mids]
        for mid in mids:
            motifs.append([mid] + data)
motifs = pd.DataFrame(motifs, columns=[h[-1]] + h[:-1])
motifs.set_index('Motif_ID', inplace=True)

In [4]:
DBDseqs = set()
for x in proteins['DBD_seqs']:
    x = x.split(',') #in cases of multiple DBDs
    for seq in x:
        DBDseqs.add(seq)
        
#Write DBDseqs to fasta
with open('HDSeqs.fa', 'w') as outfile:
    for DBDseq in DBDseqs:
        outfile.write('>' + DBDseq + '\n' + DBDseq + '\n')

### Align to Pfam HMM (source: https://pfam.xfam.org/family/PF00046/hmm)

`python RunAPHID.py ../Example/HomeodomainData/Homeodomain.hmm ./Example/HDSeqs.fa semiglobal`

In [5]:
#Read results into dictionary that maps the DBD sequence to its reference alignment
DBDseqs = {} #Actual DBDseq : Reference Alignment 
for seq, aln in alntools.FastaIter(fileloc='DBDMatchPos_aphid/HDSeqs.matchpos_semiglobal.fa'):
    DBDseqs[seq] = aln

### Get Motif Sequences and Alignments to Reference

In [6]:
MotifSequences = {} # MID: [DBD Sequences]
for mid in MIDs:
    #print mid
    minfo = motifs.loc[mid]
    protinfo = proteins.loc[minfo['TF_ID']]
    if type(protinfo['DBD_seqs']) == str:
        MotifSequences[mid] = protinfo['DBD_seqs'].split(',')
    else:
        MotifSequences[mid] = protinfo['DBD_seqs'][0].split(',')
    
#Get Motifs Sequences that are aligned to common Pfam reference
MotifSequences_aligned = {}
for mid, unaligned in MotifSequences.items():
    MotifSequences_aligned[mid] = [DBDseqs.get(x) for x in unaligned]

# Make DBD alignments and positional features for motif pairs with E-score overlaps

- The MotifAlignments contain the features, and outputs necessary to output into dataframes for training in R. 
- They can also be scored using existing SR models

In [7]:
Homeodomain_ReplicateThreshold = 0.5555

MotifAlignments = {} # MID Pair : {Alignment Information}

for pair, overlap in EScoreOverlaps.items():
    #Get Reference alignments for each motif
    proteinseqs_x = (pair[0], MotifSequences_aligned[pair[0]])
    proteinseqs_y = (pair[1], MotifSequences_aligned[pair[1]])
    #Align each motif to eachother
    sr_alignment = pwsaln.AlignDBDArrays(proteinseqs_x, proteinseqs_y)
    #Add E-score overlap to the alignment information
    sr_alignment['EScoreOverlap'] = overlap
    sr_alignment['EClass'] = int(overlap >= Homeodomain_ReplicateThreshold)
    MotifAlignments[pair] = sr_alignment

### Examples of information in an SR alignment 
Example: 

M1009_1.02:Berger08:Hoxa7_2668 

    vs.	

M1072_1.02:Badis09:Hoxa3_2783

In [8]:
sr_alignment = MotifAlignments[('M1009_1.02', 'M1072_1.02')]

print 'Motif Similarity (E-score Overlap):', sr_alignment['EScoreOverlap']
print 'Alignment % Amino Acid Identity:', '{:.3f}'.format(100*sr_alignment['PctID_L']) + '%'
print 'Positional Amino Acid Identity:', sr_alignment['ByPos.PctID']
print 'Positional Amino Acid Similarity:', sr_alignment['ByPos.AvgB62']

Motif Similarity (E-score Overlap): 0.637931034483
Alignment % Amino Acid Identity: 73.684%
Positional Amino Acid Identity: [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]
Positional Amino Acid Similarity: [5.0, 5.0, 0.0, 5.0, -1.0, 0.0, 7.0, 5.0, -1.0, -2.0, 5.0, -1.0, 1.0, 5.0, 4.0, 5.0, 5.0, 5.0, 6.0, 8.0, 6.0, 6.0, 5.0, 7.0, 4.0, -1.0, 5.0, -2.0, 5.0, 5.0, 3.0, 5.0, 1.0, 4.0, 1.0, -1.0, 4.0, -3.0, 4.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 11.0, 6.0, 5.0, 6.0, 5.0, 5.0, 5.0, 5.0, 2.0, 5.0, 5.0]


### Example of Scoring TF pairs using an existing model

In [9]:
#Read SR Model
HD_SRModel = srpred.ReadSRModel('../SRModels/F223_1.97d.json')

#Score Motif Alignments
SR_Scores_i = []
SR_Scores = []
for pair, sr_alignment in MotifAlignments.items():
    SR_Score, SR_Class = srpred.ScoreAlignmentResult(resultDict=sr_alignment, scoreDict=HD_SRModel)
    SR_Scores_i.append(pair)
    SR_Scores.append([sr_alignment['PctID_L'], sr_alignment['EScoreOverlap'], SR_Score, SR_Class])
SR_Scores = pd.DataFrame(SR_Scores, columns = ['AA %ID','EScoreOverlap', 'SR_Score', 'SR_Class'])
SR_Scores.index = pd.MultiIndex.from_tuples(SR_Scores_i)
SR_Scores['Class'] = 'Amb'
SR_Scores.loc[SR_Scores['EScoreOverlap'] >= Homeodomain_ReplicateThreshold, 'Class'] = 'HSim'
SR_Scores.loc[SR_Scores['EScoreOverlap'] < 0.2, 'Class'] = 'Dis'
SR_Scores.sort_values('SR_Score', ascending=False)

AA %ID  EScoreOverlap  SR_Score SR_Class Class
M1072_1.02 M1073_1.02  1.000000       0.887640  0.866518     HSim  HSim
M1009_1.02 M1010_1.02  1.000000       0.775000  0.866518     HSim  HSim
M1010_1.02 M1073_1.02  0.736842       0.623596  0.622210     HSim  HSim
           M1072_1.02  0.736842       0.597701  0.622210     HSim  HSim
M1009_1.02 M1073_1.02  0.736842       0.651685  0.622210     HSim  HSim
           M1072_1.02  0.736842       0.637931  0.622210     HSim  HSim
M0931_1.02 M1073_1.02  0.333333       0.643192  0.549141      Amb  HSim
           M1072_1.02  0.333333       0.671362  0.549141      Amb  HSim
M1007_1.02 M1073_1.02  0.526316       0.320225  0.522381      Amb   Amb
           M1072_1.02  0.526316       0.293103  0.522381      Amb   Amb
           M1010_1.02  0.526316       0.410526  0.520482      Amb   Amb
           M1009_1.02  0.526316       0.383333  0.520482      Amb   Amb
M0966_1.02 M1007_1.02  0.456140       0.000000  0.474253      Amb   Dis
           M1009_1.02  0.421053       0.000000  0.453149      Amb   Dis
           M1010_1.02  0.421053       0.000000  0.453149      Amb   Dis
           M1072_1.02  0.456140       0.005747  0.420110      Amb   Dis
           M1073_1.02  0.456140       0.005618  0.420110      Amb   Dis
M0931_1.02 M1007_1.02  0.315789       0.342723  0.406922      Amb   Amb
           M0966_1.02  0.298246       0.004695  0.404151      Amb   Dis
M0941_1.02 M1073_1.02  0.491228       0.005618  0.398389      Amb   Dis
           M1072_1.02  0.491228       0.005747  0.398389      Amb   Dis
M0931_1.02 M1010_1.02  0.333333       0.624413  0.378838      Amb  HSim
           M1009_1.02  0.333333       0.624413  0.378838      Amb  HSim
           M0941_1.02  0.385965       0.028169  0.362176      Amb   Dis
M0909_1.02 M0933_1.02  0.381818       0.042056  0.238042      Dis   Dis
M0941_1.02 M1009_1.02  0.456140       0.006098  0.230076      Dis   Dis
           M1010_1.02  0.456140       0.006098  0.230076      Dis   Dis
           M1049_1.02  0.327273       0.000000  0.199632      Dis   Dis
           M1007_1.02  0.368421       0.006098  0.183438      Dis   Dis
           M0966_1.02  0.368421       0.000000  0.155471      Dis   Dis
M0933_1.02 M0966_1.02  0.315789       0.004673  0.128590      Dis   Dis
           M1007_1.02  0.298246       0.000000  0.090370      Dis   Dis
           M1010_1.02  0.210526       0.004673  0.087494      Dis   Dis
           M1009_1.02  0.210526       0.004673  0.087494      Dis   Dis
M0909_1.02 M1009_1.02  0.315789       0.000000  0.085192      Dis   Dis
           M1010_1.02  0.315789       0.000000  0.085192      Dis   Dis
           M0966_1.02  0.280702       0.007576  0.063947      Dis   Dis
M0933_1.02 M1049_1.02  0.400000       0.009346  0.054308      Dis   Dis
M0931_1.02 M0933_1.02  0.290909       0.000000  0.054079      Dis   Dis
M0933_1.02 M0941_1.02  0.280702       0.000000  0.049819      Dis   Dis
M0909_1.02 M1007_1.02  0.263158       0.000000  0.049280      Dis   Dis
           M1049_1.02  0.381818       0.000000  0.043178      Dis   Dis
           M0941_1.02  0.280702       0.000000  0.042790      Dis   Dis
M0933_1.02 M1073_1.02  0.192982       0.004673  0.042592      Dis   Dis
           M1072_1.02  0.192982       0.004673  0.042592      Dis   Dis
M0909_1.02 M1072_1.02  0.245614       0.000000  0.036407      Dis   Dis
           M1073_1.02  0.245614       0.000000  0.036407      Dis   Dis
           M0931_1.02  0.263158       0.009390  0.023920      Dis   Dis
M0931_1.02 M1049_1.02  0.309091       0.000000 -0.027452      Dis   Dis
M1009_1.02 M1049_1.02  0.272727       0.000000 -0.027919      Dis   Dis
M1010_1.02 M1049_1.02  0.272727       0.000000 -0.027919      Dis   Dis
M1007_1.02 M1049_1.02  0.218182       0.000000 -0.036610      Dis   Dis
M1049_1.02 M1072_1.02  0.228070       0.000000 -0.039781      Dis   Dis
           M1073_1.02  0.228070       0.000000 -0.039781      Dis   Dis
M0966_1.02 M1049_1.02  0.290909       0.000000 -0.045417      Dis